In [35]:
import os
import glob
import zipfile
import pandas as pd
import time
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask_ml.linear_model import LinearRegression
from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import MinMaxScaler
from dask_ml.metrics import mean_absolute_error
#Install dask-ml

In [2]:
Path = "/home/michell21/Datasets/Ruoa_Dask/"

In [3]:
#client = Client(n_workers=8)

/home/michell21/anaconda3/envs/practicasML/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41521 instead
  warnings.warn(


In [3]:
df = dd.read_csv(Path+'*.csv')

In [6]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 10 entries, Unnamed: 0 to Rad_Avg
dtypes: object(1), float64(8), int64(1)

In [7]:
df["Temp_Avg"].describe().compute()

count    49727.000000
mean        17.478270
std          5.029615
min          0.940000
25%         15.680000
50%         19.230000
75%         24.252500
max         33.200000
Name: Temp_Avg, dtype: float64

In [8]:
df["Rad_Avg"].describe().compute()

count    50187.000000
mean       241.175908
std        334.975161
min      -1483.300000
25%          0.000000
50%         71.050000
75%        661.075000
max       1343.900000
Name: Rad_Avg, dtype: float64

In [9]:
df = df.dropna(subset=["Temp_Avg"]) #usar Rad_Avg

In [10]:
df = df.fillna(0)

In [11]:
df.head()

,Unnamed: 0,TIMESTAMP,Temp_Avg,RH_Avg,WSpeed_Avg,WSpeed_Max,WDir_Avg,Rain_Tot,Press_Avg,Rad_Avg
424,425,2015-08-18 16:00:00,32.32,62.96,2.441,0.0,197.7,0.0,805.0,255.3
445,446,2015-08-19 13:00:00,23.51,53.64,1.871,0.0,68.3,0.0,805.0,927.6
446,447,2015-08-19 14:00:00,23.10,56.84,1.544,0.0,145.7,0.0,805.0,333.9
447,448,2015-08-19 15:00:00,21.75,65.09,2.903,0.0,169.9,0.1,805.0,317.3
448,449,2015-08-19 16:00:00,21.43,63.42,2.705,0.0,193.4,0.0,805.0,269.2


In [12]:
y = df["Temp_Avg"]

In [18]:
y

Dask Series Structure:
npartitions=71
    float64
        ...
     ...   
        ...
        ...
Name: Rad_Avg, dtype: float64
Dask Name: try_loc, 213 tasks

In [13]:
X = df.copy()

In [14]:
X = X.drop(['TIMESTAMP', 'Unnamed: 0', "Temp_Avg"], axis=1)

## Scalar

In [15]:
scaler = MinMaxScaler() 
X_scaler = scaler.fit_transform(X)

In [16]:
X_scaler = X_scaler.to_dask_array()
y = y.to_dask_array()

In [16]:
#X = X.values
#y = y.values

dask.array<values, shape=(nan, 7), dtype=float64, chunksize=(nan, 7), chunktype=numpy.ndarray>

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_scaler.compute_chunk_sizes(), y.compute_chunk_sizes(), test_size = 0.3, shuffle=True)

In [19]:
X_train

dask.array<concatenate, shape=(34755, 7), dtype=float64, chunksize=(520, 7), chunktype=numpy.ndarray>

In [20]:
lr = LinearRegression()

In [21]:
with ProgressBar():
    lr.fit(X_train, y_train)

[########################################] | 100% Completed |  1.7s
[########################################] | 100% Completed |  3.8s
[########################################] | 100% Completed |  3.8s
[########################################] | 100% Completed |  3.8s
[########################################] | 100% Completed |  3.8s
[########################################] | 100% Completed |  3.8s
[########################################] | 100% Completed |  3.8s
[########################################] | 100% Completed |  3.8s
[########################################] | 100% Completed |  3.8s
[########################################] | 100% Completed |  3.9s
[########################################] | 100% Completed |  3.7s
[########################################] | 100% Completed |  3.8s
[########################################] | 100% Completed |  3.7s
[########################################] | 100% Completed |  3.7s
[########################################] | 100

In [22]:
y_pred = lr.predict(X_test)

In [23]:
y_pred

dask.array<tensordot, shape=(14952,), dtype=float64, chunksize=(224,), chunktype=numpy.ndarray>

In [25]:
y_pred_df = y_pred.to_dask_dataframe()

In [26]:
y_pred_df.head()

0    12.163484
1    13.389827
2    21.345237
3    12.719587
4    15.398447
dtype: float64

In [27]:
y_test.compute_chunk_sizes()

dask.array<concatenate, shape=(14952,), dtype=float64, chunksize=(224,), chunktype=numpy.ndarray>

In [28]:
y_test_df = y_test.to_dask_dataframe()
y_test_df.head()

0    16.69
1    13.80
2    23.10
3    16.48
4    17.46
dtype: float64

In [29]:
lr.score(X_test, y_test)

0.4020513395775993

In [31]:
mean_absolute_error(y_test, y_pred)

2.8035281432631147

## Procesamiento usando Client

In [32]:
from dask_ml.wrappers import ParallelPostFit
from dask.distributed import Client, progress

In [44]:
client = Client(processes=False, threads_per_worker=8,
                n_workers=4, memory_limit='8GB')
client

/home/michell21/anaconda3/envs/practicasML/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36431 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://132.248.203.7:36431/status,
Dashboard: http://132.248.203.7:36431/status,Workers: 4
Total threads: 32,Total memory: 29.80 GiB
Status: running,Using processes: False
Comm: inproc://132.248.203.7/96456/206,Workers: 4
Dashboard: http://132.248.203.7:36431/status,Total threads: 32
Started: Just now,Total memory: 29.80 GiB
Comm: inproc://132.248.203.7/96456/209,Total threads: 8
Dashboard: http://132.248.203.7:42007/status,Memory: 7.45 GiB
Nanny: None,


In [45]:
clf = ParallelPostFit(LinearRegression())

In [46]:
with ProgressBar():
    clf.fit(X_train, y_train)

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7fe81f508b50>>, <Task finished name='Task-266377' coro=<Cluster._sync_cluster_info() done, defined at /home/michell21/anaconda3/envs/practicasML/lib/python3.9/site-packages/distributed/deploy/cluster.py:98> exception=OSError('Timed out trying to connect to inproc://132.248.203.7/96456/167 after 30 s')>)
Traceback (most recent call last):
  File "/home/michell21/anaconda3/envs/practicasML/lib/python3.9/site-packages/distributed/comm/inproc.py", line 312, in connect
    await conn_req.conn_event.wait()
  File "/home/michell21/anaconda3/envs/practicasML/lib/python3.9/asyncio/locks.py", line 226, in wait
    await fut
asyncio.exceptions.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/michell21/anaconda3/envs/practicasML/lib/python3.9/asyncio/tasks

KeyboardInterrupt: 

In [23]:
clf.score(X_test, y_test)

0.4037220555294101